In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Activation,Dropout,Conv2D, MaxPooling2D,BatchNormalization
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model
# pprevent annoying tensorflow warning
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
import warnings
warnings.simplefilter("ignore")
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
import glob
import os
from PIL import Image
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Activation,Dropout,Conv2D, MaxPooling2D,BatchNormalization,Flatten
import glob
from pathlib import Path
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
TRAIN_PATH='../input/vietnamese-foods/Images/Train'
TEST_PATH='../input/vietnamese-foods/Images/Test'
IMAGE_SIZE=(224,224)
BATCH_SIZE=128 #128
VALIDATE_PATH='../input/vietnamese-foods/Images/Validate'
class_count=30
        
        
        

In [ ]:
train_generator = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 40, 
    width_shift_range = 0.2, 
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True)

validate_generator = ImageDataGenerator(rescale=1./255)
test_generator = ImageDataGenerator(rescale=1./255)

train_data = train_generator.flow_from_directory(TRAIN_PATH, target_size=IMAGE_SIZE, batch_size=BATCH_SIZE)
validate_data = validate_generator.flow_from_directory(VALIDATE_PATH, target_size=IMAGE_SIZE, batch_size=BATCH_SIZE)
test_data = test_generator.flow_from_directory(TEST_PATH, target_size=IMAGE_SIZE)

In [ ]:
# create 2 useful callbacks, one to control the learning rate, and one to control early stopping based on validation loss
rlronp=tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2,verbose=1)
# estop=tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=4, verbose=1,restore_best_weights=True)
# #rlronp,
# callbacks=[estop]

In [ ]:
pretrained_model = VGG19(weights='imagenet', include_top=False)
last_output = pretrained_model.output
x = GlobalAveragePooling2D()(last_output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.4)(x)
outputs = Dense(30, activation='softmax')(x)
model = Model(inputs=pretrained_model.input, outputs=outputs)

In [ ]:
for layer in pretrained_model.layers: layer.trainable = False #,
model.compile(optimizer='rmsprop',loss='mse', metrics=['accuracy']) 

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1)

In [ ]:
# train the model
epochs=20
history=model.fit(x=train_data ,epochs=epochs, verbose=1,validation_data=validate_data,
                  callbacks = [rlronp, early_stopping],shuffle=False, initial_epoch=0)

In [ ]:
# vẽ lại quá trình học
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epochs')
plt.legend(['train','Validation'])
plt.show()

In [ ]:
from tensorflow.keras.utils import load_img, img_to_array
import numpy as np
import matplotlib.pyplot as plt
filename = "../input/fruitdataset/train/tomato/Image_11.jpg"

img = load_img(filename,target_size=(224,224))
img_show = plt.imshow(img)
plt.show()
img = img_to_array(img)
img = img.reshape(1,224,224,3)
img = img.astype('float32')
img = img/255
kq=np.argmax(model.predict(img),axis=-1)
if(kq==0):
    print("Banh beo")
if(kq==1):
    print("Bot loc")
if(kq==2):
    print("Banh Can")
if(kq==3):
    print("Banh canh")
if(kq==4):
    print("Banh chung")
if(kq==5):
    print("Banh cuon")
if(kq==6):
    print("Banh duc")
if(kq==7):
    print("Banh gio")
if(kq==8):
    print("Banh khot")
if(kq==9):
    print("Banh mi")
if(kq==10):
    print("Banh pia")
